In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# =========================================================
# LOAD DATA (long-format table)
# =========================================================
csv_path = r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\curves_data_fin.csv"  
df = pd.read_csv(csv_path)

# Drop index column if present
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Ensure correct dtypes
df["year"] = df["year"].astype(str)
df["zone"] = pd.to_numeric(df["zone"], errors="coerce").astype(int)
df["day"] = pd.to_numeric(df["day"], errors="coerce")
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Filter invalid values
df.loc[(df["value"] < 0) | (df["value"] > 0.6), "value"] = pd.NA
df = df.dropna(subset=["zone", "day", "value"]).copy()

# =========================================================
# SETTINGS
# =========================================================
window = 5  # 5-day moving average (centered)

colors = {
    "1900": "black",
    "2000": "red",
    "2003": "green",
    "2012": "darkorange",
    "2019": "deepskyblue"
}

year_labels = {
    "1900": "1984–1999 baseline",
    "2000": "2000",
    "2003": "2003",
    "2012": "2012",
    "2019": "2019"
}

zone_titles = {
    1: "Pecný-Břidličná",
    2: "Jelení hřbet",
    3: "Velký Máj",
    4: "Vysoká hole"
}

thr_dead = 0.4
thr_dmg = 0.5

# Figure size: 17 cm wide, 600 dpi
width_in = 17 / 2.54
height_in = width_in
dpi = 600

base_fs = 8
title_fs = 10

plt.rcParams.update({
    "font.size": base_fs,
    "axes.titlesize": title_fs,
    "axes.labelsize": base_fs,
    "xtick.labelsize": base_fs,
    "ytick.labelsize": base_fs,
    "legend.fontsize": base_fs,
    "legend.title_fontsize": base_fs,
})

# =========================================================
# CREATE FIGURE
# =========================================================
fig, axes = plt.subplots(
    2, 2,
    figsize=(width_in, height_in),
    sharey=True,
    dpi=dpi
)
axes = axes.flatten()
panel_letters = ["a", "b", "c", "d"]

year_handles = {}
threshold_handles = {}

# =========================================================
# PLOTTING
# =========================================================
for i, zone in enumerate([1, 2, 3, 4]):
    ax = axes[i]
    z = df[df["zone"] == zone]

    summary = (
        z.groupby(["year", "day"])["value"]
        .agg(
            mean="mean",
            p25=lambda x: x.quantile(0.25),
            p75=lambda x: x.quantile(0.75),
        )
        .reset_index()
        .sort_values(["year", "day"])
    )

    for year in sorted(summary["year"].unique()):
        d = summary[summary["year"] == year]

        x = d["day"].to_numpy()
        y_mean = d["mean"].rolling(window, center=True, min_periods=1).mean()
        y_p25 = d["p25"].rolling(window, center=True, min_periods=1).mean()
        y_p75 = d["p75"].rolling(window, center=True, min_periods=1).mean()

        color = colors.get(year, "gray")
        (line,) = ax.plot(x, y_mean, color=color, lw=1.0,
                          label=year_labels.get(year, year))
        ax.fill_between(x, y_p25, y_p75, color=color, alpha=0.2, lw=0)

        if year not in year_handles:
            year_handles[year] = line

    # Thresholds
    h_dead = ax.axhline(thr_dead, color="black", ls="--", lw=0.8)
    h_dmg  = ax.axhline(thr_dmg, color="red",   ls="--", lw=0.8)

    threshold_handles.setdefault("Dead < 0.4", h_dead)
    threshold_handles.setdefault("Damaged 0.4–0.5", h_dmg)

    # Titles and labels
    ax.set_title(zone_titles[zone])
    ax.set_xlabel("Day of Growing Season")
    if i % 2 == 0:
        ax.set_ylabel("Mean NBR")

    ax.grid(True, lw=0.4, alpha=0.5)

    # Panel letter
    ax.text(
        0.98, 0.98, panel_letters[i],
        transform=ax.transAxes,
        ha="right", va="top",
        fontsize=base_fs, fontweight="bold"
    )

# =========================================================
# LAYOUT ADJUSTMENT
# =========================================================
fig.subplots_adjust(
    bottom=0.30,
    left=0.08,
    right=0.98,
    top=0.95,
    wspace=0.15,
    hspace=0.35
)

# =========================================================
# LEGENDS
# =========================================================
# Year legend: baseline left + affected years (2×2)
baseline_handle = [year_handles["1900"]]
affected_order = ["2000", "2003", "2012", "2019"]
affected_handles = [year_handles[y] for y in affected_order]

fig.text(0.5, 0.18, "Year", ha="center", va="center", fontsize=base_fs)

fig.legend(
    baseline_handle,
    [year_labels["1900"]],
    loc="lower center",
    bbox_to_anchor=(0.25, 0.12),
    frameon=False,
    handlelength=2.0
)

fig.legend(
    affected_handles,
    [year_labels[y] for y in affected_order],
    loc="lower center",
    bbox_to_anchor=(0.64, 0.12),
    ncol=2,
    frameon=False,
    handlelength=2.0,
    columnspacing=1.5
)

# Threshold legend
fig.legend(
    [threshold_handles["Dead < 0.4"],
     threshold_handles["Damaged 0.4–0.5"]],
    ["Dead < 0.4", "Damaged 0.4–0.5"],
    title="NBR thresholds",
    loc="lower center",
    bbox_to_anchor=(0.5, 0.03),
    ncol=2,
    frameon=False,
    handlelength=2.0,
    columnspacing=1.5
)

fig.savefig(r"U:\Paper_1\Fig8_NBR_seasonal_curves.png", dpi=600, bbox_inches="tight")
plt.show()



